In [12]:
import tensorflow as tf
import tensorflow_datasets as tfds
import mlflow
import mlflow.tensorflow
import matplotlib.pyplot as plt
import numpy as np


In [17]:
v = '01'
# MLflow konfigurieren
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(f"nn-cat-dog-classifier-{v}")
mlflow.tensorflow.autolog()

In [18]:
# Daten laden
img_height, img_width = 128, 128
batch_size = 32


In [19]:
def preprocess(image, label):
    image = tf.image.resize(image, [img_height, img_width])
    image = image / 255.0  # Normalisierung
    return image, label

In [20]:
# Train- und Validation-Daten vorbereiten
(ds_train, ds_val), ds_info = tfds.load(
    "cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
    with_info=True
)

In [21]:
ds_train = ds_train.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [22]:
# Modellarchitektur definieren
def create_model(conv_layers=2, dense_units=128, dropout_rate=0.5):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(img_height, img_width, 3))
    ])
    
    # Convolutional Layers
    for _ in range(conv_layers):
        model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))

    # Fully Connected Layer
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(dense_units, activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # Binary-Klassifikation
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [23]:
# Modelltraining mit verschiedenen Hyperparametern
param_grid = [
    {'conv_layers': 2, 'dense_units': 128, 'dropout_rate': 0.5},
    {'conv_layers': 3, 'dense_units': 256, 'dropout_rate': 0.3},
    {'conv_layers': 2, 'dense_units': 64, 'dropout_rate': 0.5},
    {'conv_layers': 4, 'dense_units': 256, 'dropout_rate': 0.2}
]
#param_grid = [{'conv_layers': 5, 'dense_units': 256, 'dropout_rate': 0.2}]

for params in param_grid:
    with mlflow.start_run():
        model = create_model(**params)
        history = model.fit(
            ds_train,
            validation_data=ds_val,
            epochs=5,
            verbose=1
        )


2024-10-13 14:57:23.080708: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/5
581/582 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5830 - loss: 0.6906

582/582 ━━━━━━━━━━━━━━━━━━━━ 38s 65ms/step - accuracy: 0.5832 - loss: 0.6905 - val_accuracy: 0.6821 - val_loss: 0.6039
Epoch 2/5
581/582 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.7046 - loss: 0.5790

582/582 ━━━━━━━━━━━━━━━━━━━━ 39s 67ms/step - accuracy: 0.7047 - loss: 0.5789 - val_accuracy: 0.7291 - val_loss: 0.5485
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 41s 71ms/step - accuracy: 0.7677 - loss: 0.4824 - val_accuracy: 0.7371 - val_loss: 0.5489
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 42s 72ms/step - accuracy: 0.8220 - loss: 0.3874 - val_accuracy: 0.7436 - val_loss: 0.5797
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 42s 72ms/step - accuracy: 0.8728 - loss: 0.3026 - val_accuracy: 0.7423 - val_loss: 0.6684


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


...

582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 60ms/step - accuracy: 0.7459 - loss: 0.5226 - val_accuracy: 0.7913 - val_loss: 0.4460
Epoch 3/5
581/582 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8005 - loss: 0.4378

582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 61ms/step - accuracy: 0.8005 - loss: 0.4377 - val_accuracy: 0.8067 - val_loss: 0.4159
Epoch 4/5
581/582 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8282 - loss: 0.3837

582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 60ms/step - accuracy: 0.8282 - loss: 0.3836 - val_accuracy: 0.8321 - val_loss: 0.3760
Epoch 5/5
581/582 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8538 - loss: 0.3358

582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 60ms/step - accuracy: 0.8538 - loss: 0.3358 - val_accuracy: 0.8388 - val_loss: 0.3643
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


2024/10/13 15:10:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run intrigued-moth-151 at: http://localhost:5000/#/experiments/848365883029898366/runs/89d4913469654aefa7b5dc9015648371.
2024/10/13 15:10:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/848365883029898366.


In [22]:
## Notwendige Imports, falls noch nicht vorhanden
#import matplotlib.pyplot as plt
#import numpy as np

# Eine Funktion zur Anzeige eines Bildes und zur Ausgabe der Modellvorhersage
def predict_and_plot_sample(model, ds_val, num_samples=20):
    for image, label in ds_val.take(num_samples):
        # Bildvorverarbeitung (falls erforderlich) und Plotten des Bildes
        plt.imshow(image[0].numpy())  # Bild als numpy array extrahieren
        plt.axis("off")
        plt.show()

        # Modellvorhersage
        prediction = model.predict(tf.expand_dims(image[0], axis=0))  # Hinzufügen einer Batch-Dimension
        prediction_label = "Dog" if prediction[0][0] > 0.5 else "Cat"
        p = 100 * (1 - prediction[0][0])
        # Vorhersage und tatsächliches Label ausgeben
      
        print(f"Model Prediction: {prediction_label} | Actual Label: {'Dog' if label[0] == 1 else 'Cat'} - {p:.2f} %")

In [10]:
predict_and_plot_sample(model, ds_val)